In [1]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_helper import rel_slicer
from PW_explorer.time_series import PWETimeSeriesModule
from PW_explorer.pwe_nb_helper import ASPRules
from PW_explorer.export import PWEExport
from PW_explorer.visualize import PWEVisualization

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from nxpd import draw
from nxpd import nxpdParams
nxpdParams['show'] = 'ipynb'
from copy import deepcopy

In [2]:
%load_ext PWE_NB_Extension

In [3]:
# type brighntess population


In [4]:
%%dlv --donot-run --donot-display_input -lci edb_instance

emp("Jones",   30000, 35, "Accounting").
emp("Miller",  38000, 29, "Marketing").
emp("Koch",  2000000, 24, "IT").
emp("Nguyen",  35000, 42, "Marketing").
emp("Gruber",  32000, 39, "IT").

dept("IT",         "Atlanta").
dept("Marketing",  "New York").
dept("Accounting", "Los Angeles").



In [5]:
%%dlv --donot-run -lci ebd_instance --donot-display_input

emp("Sun",100,1).
emp("Moon",10,2).
emp("Planet",1,8).

limit(1..10).
%q(Ename,Emag,Epop,X) :- emp(Ename,Emag,Epop), Epop > limit(X).
n(X) :- limit(X).



In [6]:
%%dlv --donot-run -lci dlv_rules --donot-display_input

q(Ename,Emag,Epop,X) :- emp(Ename,Emag,Epop), n(X), Epop > X.


%q1(Ename,Emag,Epop,X) :- q("Planet",1,8,X).

In [7]:
%dlv -l dlv_rules ebd_instance -wfs --save_meta_data_to meta_data --saveto dlv_soln --donot-display_input


Output:


'True: {emp("Sun",100,1), emp("Moon",10,2), emp("Planet",1,8), limit(1), limit(2), limit(3), limit(4), limit(5), limit(6), limit(7), limit(8), limit(9), limit(10), q("Moon",10,2,1), q("Planet",1,8,1), q("Planet",1,8,2), q("Planet",1,8,3), q("Planet",1,8,4), q("Planet",1,8,5), q("Planet",1,8,6), q("Planet",1,8,7), n(1), n(2), n(3), n(4), n(5), n(6), n(7), n(8), n(9), n(10)}\nUndefined: {}'

In [155]:
%dlv -l ebd_instance -wfs --save_meta_data_to meta_data --saveto dlv_soln --donot-display_input


Output:


'True: {emp("Sun",100,1), emp("Moon",10,2), emp("Planet",1,8), limit(1), limit(2), limit(3), limit(4), limit(5), limit(6), limit(7), limit(8), limit(9), limit(10), n(1), n(2), n(3), n(4), n(5), n(6), n(7), n(8), n(9), n(10)}\nUndefined: {}'

In [24]:
%%dlv --donot-run -lci ebd_instance --donot-display_input

data("Sun",100,1).
data("Sun",1000,10).
data("Moon",10,2).
data("Planet",1,8).

limit(1..10).
n(X) :- limit(X).


In [27]:
%%dlv --donot-run -lci dlv_rules --donot-display_input

%q(Ename,Emag,Epop,X) :- data(Ename,Emag,Epop), n(X), Epop > X.

q2(U,V,W) :- data(U,V,W), U = "Sun", .
%q1(Ename,Emag,Epop,X) :- q("Planet",1,8,X).

In [28]:
%dlv -l dlv_rules ebd_instance -wfs --save_meta_data_to meta_data --saveto dlv_soln --donot-display_input


Output:


'True: {data("Sun",100,1), data("Sun",1000,10), data("Moon",10,2), data("Planet",1,8), limit(1), limit(2), limit(3), limit(4), limit(5), limit(6), limit(7), limit(8), limit(9), limit(10), q2("Sun",100,1), q2("Sun",1000,10), n(1), n(2), n(3), n(4), n(5), n(6), n(7), n(8), n(9), n(10)}\nUndefined: {}'

In [4]:
%%clingo -l hamming_gv_rules --donot-display_input --save_meta_data_to md --saveto sail_out

data("Sun",100,1).
data("Moon",10,2).
data("Planet",1,8).

query1(X,Y,Z) :- data(X,Y,Z).
join1(X,Y,Z,A,B,C) :- query1(X,Y,Z), query1(A,B,C).

Output:


'Answer: 1\ndata("Sun",100,1) data("Moon",10,2) data("Planet",1,8) query1("Sun",100,1) query1("Moon",10,2) query1("Planet",1,8) join1("Sun",100,1,"Sun",100,1) join1("Moon",10,2,"Sun",100,1) join1("Planet",1,8,"Sun",100,1) join1("Sun",100,1,"Moon",10,2) join1("Moon",10,2,"Moon",10,2) join1("Planet",1,8,"Moon",10,2) join1("Sun",100,1,"Planet",1,8) join1("Moon",10,2,"Planet",1,8) join1("Planet",1,8,"Planet",1,8)\nSATISFIABLE\n\nModels       : 1\nCalls        : 1\nTime         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)\nCPU Time     : 0.004s'

In [5]:
pw_rel_dfs, schemas, pw_objs = load_worlds(sail_out, md, print_parse_tree=False, reasoner='clingo')

Number of Models: 1


In [7]:
pw_rel_dfs.keys()

dict_keys(['data_3', 'query1_3', 'join1_6'])

In [33]:
g = PWEVisualization.graphviz_from_meta_data(pw_rel_dfs, md['graphviz'])
draw(g)

In [8]:
pw_rels_dfs, rel_schemas, pw_objs  = load_worlds(asp_output=dlv_soln, meta_data=meta_data, reasoner='dlv')

In [9]:
list(map(lambda x: x.__dict__, rel_schemas))


[{'relation_name': 'q_4', 'arity': 4, 'r_id': 0, 'meta_data': {}},
 {'relation_name': 'limit_1', 'arity': 1, 'r_id': 1, 'meta_data': {}},
 {'relation_name': 'n_1', 'arity': 1, 'r_id': 2, 'meta_data': {}},
 {'relation_name': 'emp_3', 'arity': 3, 'r_id': 3, 'meta_data': {}}]

In [10]:
pw_rels_dfs.keys()

dict_keys(['q_4', 'limit_1', 'n_1', 'emp_3'])

In [11]:
pw_rels_dfs['q_4']

,wfs_status,x1,x2,x3,x4
0,true,"""Moon""",10,2,1
1,true,"""Planet""",1,8,1
2,true,"""Planet""",1,8,2
3,true,"""Planet""",1,8,3
4,true,"""Planet""",1,8,4
5,true,"""Planet""",1,8,5
6,true,"""Planet""",1,8,6
7,true,"""Planet""",1,8,7
